In [1]:
import SigVarGen as svg
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
def generate_single_signal(domain):
    """
    Generate a single signal for a given domain, including base signal and interrupts.
    """
    num = random.uniform(0.3,0.7)
    drop=False
    signal_range, interrupt_range = svg.generate_device_parameters(svg.EMBEDDED_DEVICE_RANGES, drop=drop, frequency_follows_amplitude=False, split_ratio=num)

    amplitude_range = signal_range[domain]['amplitude']
    frequency_range = signal_range[domain]['frequency']

    if isinstance(frequency_range, dict):
        temp = random.choice(list(frequency_range.keys()))
        frequency_range = frequency_range[temp]
    else:
        temp = 0

    t = np.linspace(0, 1, 1000)
    n_sinusoids = random.randint(50, 100)

    disperse = random.choice([True, False])
    complex_iter = random.randint(0,3)

    base_wave, sinusoids_params = svg.generate_signal(t, n_sinusoids, amplitude_range, frequency_range)

    wave, interrupt_params = svg.add_interrupt_with_params(t, base_wave.copy(), domain, signal_range, interrupt_range, 
                                            temp, disperse=disperse, drop=drop, complex_iter=complex_iter)

    #plt.figure(figsize=(50, 5))
    #plt.plot(range(len(wave)), wave, label='wave')
    #plt.legend(fontsize=18)
    #plt.show()

    return t, wave, signal_range, interrupt_range, n_sinusoids, interrupt_params, temp

In [3]:
# Main data generation loop
data_list = []
num_variations = 10  # Number of variations for each signal
color = 'white'

for j in range(100):

    domain = random.choice(list(svg.EMBEDDED_DEVICE_RANGES.keys()))
    _, wave, signal_range, interrupt_range, _, _, _ = generate_single_signal(domain)

    noise_power, _ = svg.generate_noise_power(wave, snr_range=(-10, 20))

    data = {
        'domain': domain,
        'Stationarity': "Non-Stationary Power",
        'color' : color,
        'noise_power_σ^2': noise_power,
        'signal_range': signal_range[domain],
        'response_range': interrupt_range[domain]
    }

    data['num'] = j
    data['wave'] = wave

    mod_envelope = random.choice(svg.noise_funcs)

    if mod_envelope is not None:
        mod_envelope['param'][0] = random.uniform(mod_envelope['param'][0], mod_envelope['param'][1])
        mod_envelope['param'][1] = random.uniform(mod_envelope['param'][0], mod_envelope['param'][1])
        data['mod_envelope'] = mod_envelope['func'].__name__
    else:
        data['mod_envelope'] = None

    #print(color, domain, mod_envelope)

    for nv in range(num_variations):
        # Generate variations
        variation_wave = wave.copy()

        # Add noise
        noisy_wave = svg.add_colored_noise(variation_wave, noise_power, npw=[1,1], mf=[1,1], color=color, mod_envelope=mod_envelope)

        data[f'noisy_wave_{nv}'] = noisy_wave

        #plt.figure(figsize=(50, 5))
        #plt.plot(range(len(noisy_wave)), noisy_wave, label='noise', linewidth=1)
        #plt.plot(range(len(wave)), wave, label='wave', linewidth=5)
        #plt.title(f"noisy_wave", fontsize=22)
        #plt.legend(fontsize=18)
        #plt.show()

    data_list.append(data)

df = pd.DataFrame(data_list)

(376, 462)
(56, 70)
(298, 312)
(723, 807)
(336, 350)
(358, 372)
(792, 905)
(270, 339)
(134, 204)
(280, 330)
(729, 779)
(687, 755)
(277, 285)
(194, 272)
(893, 990)
(766, 800)
(105, 209)
(263, 371)
(678, 790)
(430, 459)
(855, 884)
(118, 206)
(281, 304)
(800, 823)
(833, 945)
(292, 367)
(560, 666)
(44, 79)
(892, 992)
(222, 284)
(594, 660)
(792, 887)
(147, 214)
(326, 441)
(122, 207)
(216, 289)
(111, 217)
(637, 737)
(176, 224)
(698, 808)
(560, 633)
(671, 782)
(823, 859)
(593, 629)
(816, 919)
(722, 806)
(28, 67)
(337, 409)
(853, 915)
(654, 741)
(466, 574)
(362, 429)
(153, 256)
(628, 681)
(483, 536)
(513, 616)
(430, 497)
(837, 859)
(359, 469)
(236, 324)
(654, 752)
(57, 99)
(274, 351)
(591, 610)
(344, 453)
(514, 607)
(626, 692)
(475, 578)
(87, 165)
(219, 297)
(703, 795)
(918, 954)
(822, 858)
(506, 591)
(451, 465)
(110, 124)
(8, 86)
(867, 905)
(782, 820)
(283, 361)
(223, 239)
(755, 771)
(845, 937)
(248, 296)
(380, 492)
(235, 296)
(599, 660)
(36, 115)
(283, 292)
(578, 666)
(97, 173)
(754, 842)
(2

In [4]:
df

,domain,Stationarity,color,noise_power_σ^2,signal_range,response_range,num,wave,mod_envelope,noisy_wave_0,noisy_wave_1,noisy_wave_2,noisy_wave_3,noisy_wave_4,noisy_wave_5,noisy_wave_6,noisy_wave_7,noisy_wave_8,noisy_wave_9
0,Arduino Board,Non-Stationary Power,white,2.234573e-02,"{'amplitude': (0, 2.216077331282248), 'frequen...","{'amplitude': (2.216077331282248, 5), 'frequen...",0,"[1.675646322495731, 1.1205082144223806, 0.5671...",envelope_blockwise,"([2.8607800719704537, 0.9573428749926534, 0.54...","([1.3193566931277374, 0.5945509853372584, -0.3...","([1.4219457783096228, 2.078136058463007, 0.761...","([1.5146473959442652, 1.8025693584689695, 0.75...","([1.4130077637654415, 0.345118613706898, -0.48...","([1.7874461426289219, 1.2751647422502024, -0.0...","([1.6752058133207868, 1.1427761558085636, 0.54...","([1.6413172451573383, 1.3879983752013163, 1.35...","([1.5795687066539927, 1.205107502325502, 0.385...","([1.6957029134845238, 1.1097825355677848, 0.52..."
1,Cameras,Non-Stationary Power,white,3.464844e-04,"{'amplitude': (0, 0.4165919029295007), 'freque...","{'amplitude': (0.4165919029295007, 1), 'freque...",1,"[0.14411672812943827, 0.19608340029984572, 0.2...",envelope_sine,"([0.14237820256937214, 0.18220703653386464, 0....","([0.14411245905778683, 0.1955411367940725, 0.2...","([0.1391904412777062, 0.19077486930374676, 0.2...","([0.14419074164336904, 0.19619922398562162, 0....","([0.14428985431757707, 0.19467538400091855, 0....","([0.13107342413396808, 0.18721558286081333, 0....","([0.14440953029377887, 0.19626832816305456, 0....","([0.1434646022169737, 0.18710056130381772, 0.2...","([0.1341031785120541, 0.2077257736307321, 0.25...","([0.14499770172338927, 0.19575260398577413, 0...."
2,Smart Watches,Non-Stationary Power,white,1.024126e-07,"{'amplitude': (0, 0.0002800215511658297), 'fre...","{'amplitude': (0.0002800215511658297, 0.0009),...",2,"[0.00014861693126035595, 0.0001393048501283988...",envelope_random_walk,"([0.0001487244233621297, 0.0001397650539233725...","([0.00014860580950344402, 0.000139319761444095...","([0.00014854212438482152, 0.000139317069645315...","([0.0001484775233041642, 0.0001393039274203498...","([0.00014876832855022793, 0.000139324926501701...","([0.00014861464785189215, 0.000138990343988489...","([0.00014839839106968133, 0.000139279205932225...","([0.0001487245454843005, 0.0001393213773003856...","([0.00014871758934240228, 0.000139307764473306...","([0.00014865417647408092, 0.000139297001268194..."
3,Drones,Non-Stationary Power,white,3.339743e-04,"{'amplitude': (0, 0.589169078733192), 'frequen...","{'amplitude': (0.589169078733192, 1), 'frequen...",3,"[0.2644811406021378, 0.22646091403299456, 0.41...",None,"([0.2713541456418518, 0.22738173089360142, 0.3...","([0.23926460316167233, 0.22357217665352222, 0....","([0.2702708433257475, 0.2182537961128715, 0.40...","([0.2634127568853149, 0.23769758889079773, 0.4...","([0.233891203808856, 0.23608091701184356, 0.43...","([0.2773009454472513, 0.19078339924822849, 0.3...","([0.31491947673497933, 0.20063005437188342, 0....","([0.24176722871767653, 0.23696197248110207, 0....","([0.2649558799192369, 0.22711400004537485, 0.4...","([0.2955396233565157, 0.22231945710591863, 0.4..."
4,Home Automation Devices,Non-Stationary Power,white,4.224847e-10,"{'amplitude': (0, 0.0004554066969689704), 'fre...","{'amplitude': (0.0004554066969689704, 0.0009),...",4,"[0.00032394196194888243, 0.0002454211612745594...",None,"([0.0003192491838362713, 0.0002185292738720716...","([0.0003182916610173095, 0.0002404967189663146...","([0.0003484334081478357, 0.0002655673315740322...","([0.0003372556503881696, 0.0002243149267723303...","([0.0002804501921598036, 0.0002791392706359299...","([0.00031513962311942306, 0.000266033356150979...","([0.0003288965279666576, 0.0001984433998321579...","([0.00032710852047141906, 0.000204375952478492...","([0.00035012435313493796, 0.000212991090666848...","([0.00030050659240814085, 0.000237642962626842..."
...,...,...,...,...,...,...,...,...,...,...,..

In [5]:
df.to_csv(r'../datasets/ESdevices_nonstatpower_whitenoise_short.csv')